## 1. Business Understanding

### Project Background
Film production is a high-risk, capital-intensive industry. For a new movie studio, selecting projects with strong financial performance is essential to minimizing risk and ensuring sustainable growth.

### Real-World Problem
The studio needs to decide which films to produce by identifying those with:
- High global gross profit and efficient budget utilization (high ROI).
- Favorable trends in market performance over time.
- Successful genres and proven talent (directors and actors).

### Stakeholders
- **Studio Management & Investors:** Require data-driven recommendations to allocate resources effectively.
- **Production Teams:** Benefit by focusing on film attributes that historically correlate with success.

This analysis provides a clear roadmap, enabling stakeholders to make informed decisions that address a real-world business challenge.


## 2. Data Understanding

### Data Sources
1. **Movie Budgets Data:**  
   - Contains production budgets, domestic and worldwide gross revenue, and release dates for films (post-2000).
   - Offers insights into financial performance.
   
2. **IMDb Data:**  
   - Provides film metadata including ratings, runtime, genres, and details about key personnel (actors, directors, etc.).
   - Comes from a SQLite database (and a supplemental CSV) that enriches our analysis.

### Data Properties and Relevance
- **Temporal Coverage:** Films released from 2000 onward.
- **Key Features:** Budget, revenues, profit, ROI, runtime, genres, cast/crew details.
- **Utility:** The financial data combined with IMDb metadata allows us to identify trends and evaluate the efficiency of budget usage across films.
- **Limitations:** Early encoding issues and missing values have been handled during data preparation to ensure data reliability.

Confirms that the data sources are well-suited to address the studio’s problem of identifying films with high potential.

## 3. Data Preparation

### Overview
In this section, we load, clean, and merge the raw datasets to create a master dataset for analysis. Steps include:
- Parsing dates and filtering films released from 2000 onward.
- Converting currency strings to floats.
- Fixing text encoding issues.
- Calculating Global Gross Profit and ROI.
- Merging budget data with enriched IMDb data via a composite key ("Title & Year").

In [42]:
# =============================================================================
# Environment Setup and Helper Functions (Code Quality: clear, commented, and DRY)
# =============================================================================
import os
import zipfile
import warnings
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import StrMethodFormatter

# Configure display options and suppress warnings
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.0f}'.format
warnings.simplefilter('ignore')
sns.set_palette('viridis')
plt.style.use(['ggplot', 'dark_background'])

In [43]:
def fix_text_encoding(text):
    """Fix common text encoding issues in movie titles. Analysed the file to check for the encoding issue"""
    if isinstance(text, str):
        return text.replace('â\x80\x99', "'").replace('â\x80\x94', " ").replace('Ã©', "e")
    return text

def money_to_float(money_str):
    """Convert currency strings like '$1,234' into a float."""
    return float(money_str.replace('$', '').replace(',', ''))

In [44]:
# =============================================================================
# 1. Load and Clean Budget Data
# =============================================================================
df_budget = pd.read_csv('./zippedData/tn.movie_budgets.csv.gz', 
                        parse_dates=['release_date'], encoding='utf-8')
df_budget.head(2)


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,2009-12-18,Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"


In [45]:
# Extract the release year and filter for movies from 2000 onward
df_budget['release_year'] = df_budget['release_date'].dt.year
df_budget = df_budget[df_budget['release_year'] >= 2000].copy()
df_budget.head(2)

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year
0,1,2009-12-18,Avatar,"$425,000,000","$760,507,625","$2,776,345,279",2009
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875",2011


In [46]:
# Clean monetary columns by converting strings to floats
df_budget['production_budget'] = df_budget['production_budget'].apply(money_to_float)
df_budget['domestic_gross']   = df_budget['domestic_gross'].apply(money_to_float)
df_budget['worldwide_gross']  = df_budget['worldwide_gross'].apply(money_to_float)

In [47]:
# Calculate Global Gross Profit and Global Gross ROI
df_budget['Global_Gross_Profit'] = df_budget['worldwide_gross'] - df_budget['production_budget']
df_budget['Global_Gross_ROI']    = df_budget['Global_Gross_Profit'] / df_budget['production_budget']
df_budget.head(2)

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year,Global_Gross_Profit,Global_Gross_ROI
0,1,2009-12-18,Avatar,425000000,760507625,2776345279,2009,2351345279,6
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,2011,635063875,2


In [48]:
# Drop unnecessary columns and rename for clarity
df_budget.drop(columns=['id', 'release_date'], inplace=True)
rename_dict = {'movie': 'Title', 
               'production_budget': 'Budget', 
               'domestic_gross': 'Domestic_Gross_Rev', 
               'worldwide_gross': 'Global_Gross_Rev', 
               'release_year': 'Year'}
df_budget.rename(columns=rename_dict, inplace=True)
df_budget.head(10)


,Title,Budget,Domestic_Gross_Rev,Global_Gross_Rev,Year,Global_Gross_Profit,Global_Gross_ROI
0,Avatar,425000000,760507625,2776345279,2009,2351345279,6
1,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,2011,635063875,2
2,Dark Phoenix,350000000,42762350,149762350,2019,-200237650,-1
3,Avengers: Age of Ultron,330600000,459005868,1403013963,2015,1072413963,3
4,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,2017,999721747,3
5,Star Wars Ep. VII: The Force Awakens,306000000,936662225,2053311220,2015,1747311220,6
6,Avengers: Infinity War,300000000,678815482,2048134200,2018,1748134200,6
7,Pirates of the Caribbean: At Worldâs End,300000000,309420425,963420425,2007,663420425,2
8,Justice League,300000000,229024295,655945209,2017,355945209,1
9,Spectre,300000000,200074175,879620923,2015,579620923,2


In [49]:
# Fix text encoding issues in the Title column
df_budget['Title'] = df_budget['Title'].apply(fix_text_encoding)
df_budget.head(10)

,Title,Budget,Domestic_Gross_Rev,Global_Gross_Rev,Year,Global_Gross_Profit,Global_Gross_ROI
0,Avatar,425000000,760507625,2776345279,2009,2351345279,6
1,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,2011,635063875,2
2,Dark Phoenix,350000000,42762350,149762350,2019,-200237650,-1
3,Avengers: Age of Ultron,330600000,459005868,1403013963,2015,1072413963,3
4,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,2017,999721747,3
5,Star Wars Ep. VII: The Force Awakens,306000000,936662225,2053311220,2015,1747311220,6
6,Avengers: Infinity War,300000000,678815482,2048134200,2018,1748134200,6
7,Pirates of the Caribbean: At World's End,300000000,309420425,963420425,2007,663420425,2
8,Justice League,300000000,229024295,655945209,2017,355945209,1
9,Spectre,300000000,200074175,879620923,2015,579620923,2


In [50]:
# Create composite key "Title & Year" for merging with IMDb data
df_budget['Title & Year'] = df_budget.apply(lambda row: f"{row['Title']} - {row['Year']}", axis=1)


In [51]:
# Save target lists for filtering IMDb data
target_titles = df_budget['Title'].tolist()
target_title_years = df_budget['Title & Year'].tolist()

## 2. Load IMDb Data from SQLite and Supplemental CSV

The IMDb data provide additional film metadata including ratings, runtime, genres, and key personnel (actors, directors, etc.). We perform the following:
- Unzip and connect to the SQLite database.
- Execute an SQL query that joins multiple tables (film basics, ratings, and cast/crew information).
- Load a supplemental IMDb CSV file to enrich our data.
- Merge both sources using our composite key ("Title & Year") to ensure consistency with the budget data.
